In [ ]:
# Data Cleaning Workflow Template

This notebook is a reusable template for cleaning raw data:
1. Load data
2. Inspect structure
3. Handle missing values
4. Remove duplicates
5. Fix data types
6. Clean text / categories
7. Detect & cap outliers (IQR)
8. Validate & export


In [ ]:
# === 1. IMPORTS & SETTINGS ===

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

print("Libraries loaded.")


In [ ]:
# === 2. LOAD RAW DATA ===
# TODO: Change the file name/path for each new project.

DATA_PATH = "raw_data.csv"  # <-- replace with your file
df = pd.read_csv(DATA_PATH)

print("Rows:", len(df))
df.head()


In [ ]:
# === 3. BASIC STRUCTURE CHECK ===

df.info()
df.describe()


In [ ]:
# === 4. MISSING VALUES OVERVIEW ===

print("Missing values per column:")
print(df.isnull().sum())

plt.figure(figsize=(10, 4))
sns.heatmap(df.isnull(), cbar=False)
plt.title("Missing Values Heatmap")
plt.xlabel("Columns")
plt.ylabel("Rows")
plt.show()


In [ ]:
# === 5. IDENTIFY COLUMN TYPES ===

numeric_cols = df.select_dtypes(include=['float64', 'int64', 'Int64']).columns.tolist()
text_cols    = df.select_dtypes(include=['object']).columns.tolist()

print("Numeric columns:", numeric_cols)
print("Text columns:", text_cols)


In [ ]:
# === 6. HANDLE MISSING VALUES ===
# Adjust per project if needed.

# 6a. Numeric → fill with median
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        median_value = df[col].median()
        df.loc[:, col] = df[col].fillna(median_value)

# 6b. Text → fill with "Unknown"
for col in text_cols:
    if df[col].isnull().sum() > 0:
        df.loc[:, col] = df[col].fillna("Unknown")

print("Missing values after fill:")
print(df.isnull().sum())


In [ ]:
# === 7. REMOVE DUPLICATES ===

dupes_before = df.duplicated().sum()
print("Duplicates before:", dupes_before)

df = df.drop_duplicates()

dupes_after = df.duplicated().sum()
print("Duplicates after:", dupes_after)


In [ ]:
# === 8. FIX DATA TYPES ===

# Example: convert columns whose name suggests date/time
date_like_cols = [c for c in df.columns if 'date' in c.lower()]
for col in date_like_cols:
    df.loc[:, col] = pd.to_datetime(df[col], errors='coerce')

# Example: convert typical integer-like columns if present
int_candidate_cols = ['age', 'quantity', 'count']  # edit per dataset
for col in int_candidate_cols:
    if col in df.columns:
        df.loc[:, col] = df[col].astype('Int64')

print(df.dtypes)


In [ ]:
# === 9. CLEAN TEXT / CATEGORICAL COLUMNS ===

for col in text_cols:
    df.loc[:, col] = df[col].astype(str)
    df.loc[:, col] = df[col].str.strip()
    df.loc[:, col] = df[col].str.upper()

# Example: inspect a key category, if it exists
if 'CITY' in df.columns:
    print(df['CITY'].value_counts().head(20))


In [ ]:
# === 10. OUTLIER DETECTION & CAPPING (IQR) ===

def iqr_bounds(series: pd.Series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return lower, upper

for col in numeric_cols:
    if df[col].nunique() <= 1:
        continue  # skip constant columns
    
    lower, upper = iqr_bounds(df[col])
    print(f"{col}: lower={lower:.2f}, upper={upper:.2f}")
    
    df.loc[df[col] < lower, col] = lower
    df.loc[df[col] > upper, col] = upper


In [ ]:
# === 11. FINAL VALIDATION ===

print("Missing values per column:")
print(df.isnull().sum())

print("\nDuplicates:", df.duplicated().sum())

print("\nData types:")
print(df.dtypes)

print("\nPreview:")
df.head()
